In [1]:
import numpy as np
import pandas as pd

# 프로필에서 4개의 영역 및 그에 대한 총점을 얻어왔다고 가정
user_grade = pd.DataFrame(np.random.randint(1,11,size=(180,4)) , index=range(1,181), columns=['데이터분석','웹','디자인','모델링'])
user_grade_sum = user_grade.sum(axis=1)

display(user_grade)
display(user_grade_sum)

,데이터분석,웹,디자인,모델링
1,3,2,6,8
2,1,4,6,10
3,5,8,6,5
4,8,5,5,7
5,8,8,2,10
...,...,...,...,...
176,3,3,8,2
177,6,4,9,2
178,3,2,2,4
179,9,6,8,2


1      19
2      21
3      24
4      25
5      28
       ..
176    16
177    21
178    11
179    25
180    18
Length: 180, dtype: int64

In [2]:
# 실제 코드에는 쓰이지 않을 랜덤값 생성 함수

import random

def get_heart_list(x):
    heart_list = [0 for _ in range(0,x)]
    for _ in range(x):
        rand_idx = random.randint(0,x-1)
        rand_point = random.randint(0,10)
        heart_list[rand_idx] += rand_point
        if sum(heart_list) >= 10:
            heart_list[rand_idx] -= rand_point
            break
    heart_list[random.randint(0,x-1)] += 10 - sum(heart_list)
    return heart_list

def get_all_heart_list(x, y):
    all_list = []
    for _ in range(0, y):
        all_list.append(get_heart_list(x))
    return np.array(all_list)

get_all_heart_list(6, 180)

array([[ 0,  4,  0,  0,  6,  0],
       [ 0, 10,  0,  0,  0,  0],
       [ 0,  9,  0,  1,  0,  0],
       ...,
       [ 9,  0,  1,  0,  0,  0],
       [ 0,  0,  0, 10,  0,  0],
       [10,  0,  0,  0,  0,  0]])

In [3]:
# 흥미 투표 결과 데이터 얻어왔다고 가정
user_interest = pd.DataFrame(get_all_heart_list(6, 180) , index=range(1,181), columns=['건강','경제','문화/예술','교육','사회','기술'])
display(user_interest)

,건강,경제,문화/예술,교육,사회,기술
1,0,7,0,3,0,0
2,0,10,0,0,0,0
3,5,4,1,0,0,0
4,0,3,7,0,0,0
5,4,5,0,1,0,0
...,...,...,...,...,...,...
176,1,4,0,0,5,0
177,0,0,2,0,8,0
178,0,0,2,3,5,0
179,0,10,0,0,0,0


In [4]:
# 선호역할 데이터 얻어왔다고 가정
user_preference_role = pd.DataFrame(get_all_heart_list(4, 180) , index=range(1,181), columns=['데이터분석','웹','디자인','모델링'])
display(user_preference_role)

,데이터분석,웹,디자인,모델링
1,0,9,1,0
2,0,4,6,0
3,2,2,2,4
4,6,0,3,1
5,0,10,0,0
...,...,...,...,...
176,3,0,7,0
177,0,0,6,4
178,6,4,0,0
179,0,2,8,0


In [5]:
from sklearn.metrics.pairwise import cosine_similarity

# 각 유사도측정
grade_similarity = cosine_similarity(user_grade, user_grade)
interest_similarity = cosine_similarity(user_interest, user_interest)
preference_role_similarity = cosine_similarity(user_preference_role, user_preference_role)

In [6]:
# 유저의 정보 출력
# 프로필 성적 / 흥미분야 / 선호역할 순
def get_info(input_num):
    print("[프로필 성적]")
    for i in range(len(user_grade.loc[input_num])):
        print(user_grade.loc[input_num].index[i], user_grade.loc[input_num][i], end=" ")
    print("총합",user_grade_sum[input_num])
    print("[흥미 분야]")
    for interest in user_interest.loc[input_num].sort_values(ascending=False).index.tolist()[:3]:
        if user_interest.loc[input_num][interest] != 0:
            print(interest, end=" ")
    print("\n[선호 역할]")
    print(user_preference_role.loc[input_num].sort_values(ascending=False).index.tolist()[0])
get_info(180)

[프로필 성적]
데이터분석 3 웹 10 디자인 1 모델링 4 총합 18
[흥미 분야]
교육 사회 
[선호 역할]
모델링


In [10]:
input_num = int(input())
grade_subs = np.abs(user_grade_sum - user_grade_sum[input_num]) * 0.1
# 최종 평가값 'evaluation_value'
# grade_similarity(성적분포 유사도) 작을수록
# grade_subs(총점 편차) 작을수록
# interest_similarity(흥미투표 유사도) 클수록
# preference_role_similarity(선호역할 유사도) 작을수록
# 최종 평가값 'evaluation_value'가 작아진다.
evaluation_value = grade_similarity[input_num-1] + grade_subs - (interest_similarity[input_num-1]) + preference_role_similarity[input_num-1]
# 평가값이 작은 순서대로 배열 후 TOP10만 얻어온다.
top10 = evaluation_value.sort_values()[:10].index.tolist()

# 출력
print("===========내성적===========")
get_info(input_num)
for idx, value in enumerate(top10):
    print(f"===========추천도{idx}순위===========")
    get_info(value)

===========내성적===========
[프로필 성적]
데이터분석 3 웹 10 디자인 1 모델링 4 총합 18
[흥미 분야]
교육 사회 
[선호 역할]
모델링
===========추천도0순위===========
[프로필 성적]
데이터분석 9 웹 1 디자인 5 모델링 2 총합 17
[흥미 분야]
교육 문화/예술 기술 
[선호 역할]
웹
===========추천도1순위===========
[프로필 성적]
데이터분석 7 웹 1 디자인 7 모델링 5 총합 20
[흥미 분야]
교육 기술 
[선호 역할]
디자인
===========추천도2순위===========
[프로필 성적]
데이터분석 10 웹 1 디자인 4 모델링 5 총합 20
[흥미 분야]
교육 기술 
[선호 역할]
데이터분석
===========추천도3순위===========
[프로필 성적]
데이터분석 1 웹 1 디자인 9 모델링 8 총합 19
[흥미 분야]
기술 교육 
[선호 역할]
데이터분석
===========추천도4순위===========
[프로필 성적]
데이터분석 8 웹 1 디자인 7 모델링 6 총합 22
[흥미 분야]
교육 문화/예술 
[선호 역할]
데이터분석
===========추천도5순위===========
[프로필 성적]
데이터분석 8 웹 5 디자인 5 모델링 1 총합 19
[흥미 분야]
교육 
[선호 역할]
웹
===========추천도6순위===========
[프로필 성적]
데이터분석 9 웹 3 디자인 4 모델링 1 총합 17
[흥미 분야]
문화/예술 교육 
[선호 역할]
웹
===========추천도7순위===========
[프로필 성적]
데이터분석 3 웹 2 디자인 6 모델링 8 총합 19
[흥미 분야]
경제 교육 
[선호 역할]
웹
===========추천도8순위===========
[프로필 성적]
데이터분석 6 웹 2 디자인 8 모델링 3 총합 19
[흥미 분야]
경제 교육 
[선호 역할]
웹
===========추천도9순위===========
[프로필 성적]
데이터분석 6 